<a href="https://colab.research.google.com/github/mostafanabil198/Reviewlize-Graduation-Project/blob/aspect-opinion-pair-extraction/GP_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
corenlp_dir = 'drive/My Drive/GP/pair-extraction/corenlp'

## Installation & Environment Setup


### Installing Stanza

In [ ]:
# Install stanza
!pip install stanza

# Import stanza
import stanza

     |████████████████████████████████| 235kB 8.4MB/s 


### Installing Spacy and Pysbd

In [ ]:
# Install pysbd
!pip install pysbd

import spacy
from pysbd.utils import PySBDFactory
nlp = spacy.blank('en')
nlp.add_pipe(PySBDFactory(nlp))


     |████████████████████████████████| 71kB 4.7MB/s 


### Setting up Stanford CoreNLP on server

In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

2021-01-25 00:21:13 INFO: Installing CoreNLP package into ./corenlp...
2021-01-25 00:30:06 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


### Dowinloading  Stanford CoreNLP On Drive

In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = 'drive/My Drive/GP/pair-extraction/corenlp'
stanza.install_corenlp(dir=corenlp_dir)

2021-01-25 02:46:12 INFO: Installing CoreNLP package into drive/My Drive/GP/pair-extraction/corenlp...
2021-01-25 03:07:49 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=drive/My Drive/GP/pair-extraction/corenlp`.


### Set Environment variable for CoreNLP on Drive

In [ ]:
# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

## Initialize Stanford Parser CoreNLP Interface

### Constructing CoreNLPClient

In [ ]:
# Import client module
from stanza.server import CoreNLPClient

In [ ]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
        annotators=['tokenize','pos','lemma','depparse'],
        memory='16G',
        endpoint='http://localhost:9001',
        be_quiet=True)

print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

2021-01-26 13:21:38 INFO: Writing properties to tmp file: corenlp_server-d7454ca997e84c57.props
2021-01-26 13:21:38 INFO: Starting server with command: java -Xmx16G -cp drive/My Drive/GP/pair-extraction/corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-d7454ca997e84c57.props -annotators tokenize,pos,lemma,depparse -preload -outputFormat serialized


In [ ]:
# Print background processes and look for java
# You should be able to see a StanfordCoreNLPServer java process running in the background
!ps -o pid,cmd | grep java

    283 java -Xmx16G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-eea30e2be2264eea.props -annotators tokenize,pos,lemma,depparse -preload -outputFormat serialized
    306 /bin/bash -c ps -o pid,cmd | grep java
    308 grep java


# -Dependency printing

In [ ]:
document = client.annotate("I am happy with my Nokia phone")
sentence = document.sentence[0]
sentence.token[2].pos

'JJ'

In [ ]:
def getDependencies(review_sentence):
  # doc = nlp(text)
  # for t in list(doc.sents):
  document = client.annotate(str(review_sentence))
  sentence = document.sentence[0]
  dependency_parse = sentence.basicDependencies
  tokens_parse = sentence.token
  token_dict = {}
  pos_dict = {}
  for i in range(0, len(sentence.token)) :
      token_dict[sentence.token[i].tokenEndIndex] = sentence.token[i].word
      pos_dict[sentence.token[i].word] = sentence.token[i].pos

  #get a list of the dependencies with the words they connect
  list_dep=[]
  for i in range(0, len(dependency_parse.edge)):

      source_node = dependency_parse.edge[i].source
      source_name = token_dict[source_node]

      target_node = dependency_parse.edge[i].target
      target_name = token_dict[target_node]

      dep = dependency_parse.edge[i].dep
      list_dep.append((dep, source_name, target_name))
  # print(list_dep)
  return list_dep, pos_dict

In [ ]:
examples = {1: "The battery life is good",
            2: "It is great having the LCD display",
            3: "I like this camera",
            4: "The optical zoom works great",
            5: "It has movie mode that works good for a digital camera",
            6: "There is a great camera",
            71: "Nokia has fine, excellent, cheapest battery",
            72: "Nokia has fine, excellent and cheapest battery",
            8: "Nokia has good screen and battery",
            9: "I am happy with my Nokia phone",
            10: "Battery life was never a problem for me",
            11: "This camera will give you a great picture quality, LCD screen, and price"}

In [ ]:
# 1
# Extract pair when it's after capular verb => the battery is good
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def copularVerb(list_dep, list_pos, nsubjIx):
  aspect = ""
  opinion = ""
  for nsubjI in nsubjIx:
    if("JJ" in list_pos[list_dep[nsubjI][1]]):
      for dep in list_dep:
        if dep[0] == "cop" and dep[1] == list_dep[nsubjI][1]:
          opinion = list_dep[nsubjI][1]
          aspect = list_dep[nsubjI][2]
    if aspect != "" and opinion != "":
      break
  return aspect, opinion

# Call after any function to return compund aspects or same aspect => battery life
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getCompund(list_dep, aspect):
  for dep in list_dep:
    if dep[0] == "compound" and dep[1] == aspect :
      aspect = dep[2] + " " + aspect
      break;
  return aspect


In [ ]:
x, y, nsubjI = getDependencies(examples[1])
aspect, opinion = copularVerb(x, y, nsubjI)
aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ",opinions)

In [ ]:
# 3
# Extract pair when the opinions is the verb like love and like
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
# sentiment_verbs => list of verbs that may be opinion
sentiment_verbs = ["like", "love", "adore", "enjoyed", "liked", "loved", "enjoy"]
def opinionVerb(list_dep, dep):
  aspect = ""
  opinion = ""
  if(sentiment_verbs.count(dep[1])):
    opinion = dep[1]
    for dep in list_dep:
      if (dep[0] == "obj" and dep[1] == opinion):
        aspect = dep[2]
  return aspect, opinion

In [ ]:
x, y, nsubjI = getDependencies(examples[3])
aspect, opinion = opinionVerb(x, y, nsubjI)
aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

[('nsubj', 'like', 'I'), ('obj', 'like', 'camera'), ('det', 'camera', 'this')]
['camera']  -  ['like']


In [ ]:
# 5
# Extract pair when there is a relative clause relation between the aspect and the opinion => movie mode that works good
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def relativeClause(list_dep, dependency):
  aspect = dependency[1]
  relcl_opinion = dependency[2]
  opinion = ""
  for dep in list_dep:
    if dep[0] == "advmod" and relcl_opinion == dep[1]:
      opinion = dep[2]
      return aspect, opinion
  return aspect, opinion

In [ ]:
x, y, nsubjI = getDependencies(examples[5])
aspect, opinion = relativeClause(x, y, nsubjI)
aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

[('nsubj', 'has', 'It'), ('obj', 'has', 'mode'), ('compound', 'mode', 'movie'), ('acl:relcl', 'mode', 'works'), ('nsubj', 'works', 'that'), ('advmod', 'works', 'good'), ('obl', 'works', 'camera'), ('case', 'camera', 'for'), ('det', 'camera', 'a'), ('amod', 'camera', 'digital')]
['movie mode']  -  ['good']


In [ ]:
# 7-1
# Extract multi opinions if without "and" word. only with ","
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def multiOpinion(list_dep, dependency, visited_dep):
  aspect = dependency[1]
  opinions = []
  for dep in list_dep:
    if(dep[0] == "amod" and dependency[1] == dep[1]):
      opinions.append(dep[2])
      visited_dep[list_dep.index(dep)] = True
  opinions = getAndOpinions(list_dep, opinions)
  return aspect, opinions

# 7-2
# Call after any function to get list of opinions if multi exist with "and" word
# list_dep => dependency_parsed
# opnions => list of opinions "List"
def getAndOpinions(list_dep, opinions):
  for dep in list_dep:
    if dep[0] == "conj" and (opinions.count(dep[1]) != 0):
      opinions.append(dep[2])
    elif dep[0] == "conj" and (opinions.count(dep[2]) != 0):
      opinions.append(dep[1])
  return opinions

In [ ]:
x, y, nsubjI = getDependencies(examples[72])
# aspect, opinions = multiOpinion(x, y, nsubjI)
# opinions = getAndOpinions(x, [opinion])
# aspects = getAndAspects(x, aspects)
# print(aspects, " - ", opinions)

[('nsubj', 'has', 'Nokia'), ('advmod', 'has', 'fine'), ('punct', 'has', ','), ('obj', 'has', 'battery'), ('conj', 'excellent', 'cheapest'), ('cc', 'cheapest', 'and'), ('amod', 'battery', 'excellent')]


In [ ]:
# 9
# Extract pair when theres a preposition => i am happy "with" my phone
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def prepositions(list_dep, list_pos, nsubjIs):
  opinion_pos = ["JJ", "JJS", "RB"]
  aspect = ""
  opinion = ""
  for nsubjI in nsubjIs:
    if (opinion_pos.count(list_pos[list_dep[nsubjI][1]]) != 0):
      for dep in list_dep:
          if dep[0] == "obl" and dep[1] == list_dep[nsubjI][1]:
            aspect = dep[2]
            opinion = dep[1]
    if aspect != "" and opinion != "" :
      break
  return aspect, opinion

In [ ]:
x, y, nsubjI = getDependencies(examples[9])
# x, y, nsubjI = getDependencies("I had fun with my nokia phone")
aspect, opinion = prepositions(x, y, nsubjI)
# aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

[('nsubj', 'happy', 'I'), ('cop', 'happy', 'am'), ('obl', 'happy', 'phone'), ('case', 'phone', 'with'), ('nmod:poss', 'phone', 'my'), ('compound', 'phone', 'Nokia')]


'JJ'

In [ ]:
# 11
# Call after any function to get list of aspects if multi exist
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getAndAspects(list_dep, aspect):
  aspects = [aspect]
  for dep in list_dep:
    if dep[0] == "conj" and (dep[1] == aspect):
      aspects.append(dep[2])
    elif dep[0] == "conj" and (dep[2] == aspect):
      aspects.append(dep[1])
  for i in range(len(aspects)):
    aspects[i] = getCompund(list_dep, aspects[i])
  return aspects

In [ ]:
x, y, nsubjI = getDependencies(examples[11])
# x, y, nsubjI = getDependencies("I had fun with my nokia phone")
aspect, opinions = multiOpinion(x, y, nsubjI)
# aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, opinions)
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

[('det', 'camera', 'This'), ('nsubj', 'give', 'camera'), ('aux', 'give', 'will'), ('iobj', 'give', 'you'), ('obj', 'give', 'quality'), ('det', 'quality', 'a'), ('amod', 'quality', 'great'), ('compound', 'quality', 'picture'), ('punct', 'quality', ','), ('conj', 'quality', 'screen'), ('punct', 'quality', ','), ('conj', 'quality', 'price'), ('compound', 'screen', 'LCD'), ('cc', 'price', 'and')]
['picture quality', 'LCD screen', 'price']  -  ['great']


In [ ]:
# when the secomd argument in nsubj dependency is not noun and the first is adj.
# the feature of the sentence will be the object of it.
# use dependencies (nsubj , xcomp , obj) to get the feature.
# Ex : it is great having the LCD Display.
def apply_nsubj_second(list_dep, dependancy):
  f_arg = ''
  s_arg = dependancy[1]
  verb = ''
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == s_arg :
      verb = dep[2]
      break
    if dep[0] == 'dep' and dep[1] == s_arg :
      verb = dep[2]
      break
  for dep in list_dep:
    if dep[0] == 'obj' and dep[1] == verb :
      f_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg

In [ ]:
# when the second argument is noun and the opnion part is verb.
# the opinion will be the complement of the verb.
# Ex : the flash works great.
def apply_nsubj_forth (list_dep, dependency) :
  f_arg = dependency[2]
  s_arg = ''
  verb = dependency[1]
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == verb :
      s_arg = dep[2]
      break
    if dep[0] == 'advmod' and dep[1] == verb :
      s_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [ ]:
# when the feature and opinion pair candidate could in same dependency.
# amod dependency the first arg is considered to be the feature.
# the second arg is considered to be the opinion words.
# Ex : this is a great screen.
def apply_amod_sixth (list_dep, dependency) :
  f_arg = dependency[1]
  s_arg = dependency[2]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [ ]:
# when the same opinion word is used to describe more than one feature.
# apply conj dependency on the related features. 
# Ex : Nokia has a good screen and battery.
def apply_amod_eight (list_dep, feature) :
  feats = []
  for dep in list_dep:
    if dep[0] == 'conj' and dep[1] == feature:
      feats.append(dep[2])
  #call the function check compound of each feature
  return feats

In [ ]:
# when nsubj dependency is between two nouns.
# the first argument can be used as a opinion word.
# Ex : the battery was never a problem.
def apply_nsubj_ten (list_dep, dependency):
  f_arg = dependency[2]
  s_arg = dependency[1]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg
  

In [ ]:
# check negation of the opinion words as it can reverse the sentiment.
def check_neg(list_dep, adj):
    list_neg = ['no' , 'never' , 'not' , 'n\'t' , 'none' , 'neither'] 
    #can also check the negative seed list for that.
    for dep in list_dep:
      if dep[0] == 'advmod' and dep[1] == adj and ( dep[2] in list_neg) :
        adj = dep[2] + ' ' + adj
        return adj
    return adj

In [ ]:
def nsubjAdj(list_dep, list_pos, dependency):
  aspect = dependency[2]
  opinion = dependency[1]
  if list_pos[dependency[1]] == "JJR":
    for dep in list_dep:
      if dep[0] == "advcl" and dep[1] == dependency[1]:
        opinion = dep[2]
  return aspect, opinion

# -All Together


In [ ]:
def aspectOpinionPairExtractor(list_dep, list_pos):
  aspect_opinion_pairs = set()
  visited_dep = [False]*len(list_dep)
  for dep in list_dep:
    if dep[0] == 'nsubj':
      # print("nsubj")
      if "JJ" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #1
        aspect, opinion = nsubjAdj(list_dep, list_pos, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "JJ" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #2
        aspect, opinion = apply_nsubj_second(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #3
        aspect, opinion = opinionVerb(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #4
        aspect, opinion = apply_nsubj_forth(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "NN" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #10
        aspect, opinion = apply_nsubj_ten(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'amod' and visited_dep[list_dep.index(dep)] == False:
      # print("amod")
      if "JJ" in list_pos[dep[2]]:
        aspect, opinion = apply_amod_sixth(list_dep, dep) #6
        aspects = apply_amod_eight(list_dep, aspect) #8
        aspects.append(aspect)
        for a in aspects:
          addPair(a, opinion, list_dep, aspect_opinion_pairs)
        aspect, opinions = multiOpinion(list_dep, dep, visited_dep) #7
        for o in opinions:
          addPair(aspect, o, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'obl' and "JJ" in list_pos[dep[1]]: #9
      # print("obl")
      aspect, opinion = prepositions(dep)
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'acl:relcl': #5
      # print("acl:relcl")
      aspect, opinion = relativeClause(list_dep, dep)
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
  return aspect_opinion_pairs

In [ ]:
def addPair(aspect, opinion, list_dep, aspect_opinion_pairs):
  if aspect == "" or opinion == "":
    return
  aspect = getCompund(list_dep, aspect)
  opinion = check_neg(list_dep, opinion)
  p = (aspect, opinion)
  # print(p)
  aspect_opinion_pairs.add(p)

In [ ]:
aspect_opinion_pairs = set()
list_dep, list_pos = getDependencies(examples[3])
# print(list_pos["more"])
x = aspectOpinionPairExtractor(list_dep, list_pos)
print(x)

{('camera', 'like')}


# Aspect-Opinion Pair Extraction

## Reviews Anlyzer

In [ ]:
def reviewsAnalyzer(reviews_list):
  # list to hold each review's sentences and its aspects 
  # reviews->[ sentences->[ aspects->[{aspect: opinion}, {aspect, opinion}]], [[]], [[]], ]
  reviews_sentences_aspects = []

  # For each review:
    # 1- split to sentences using spacy
    # 2- for each sentence:
      # 3- get dependences
      # 4- extract aspect_opinion pairs
  
  for review in reviews_list:
    review_boundries = nlp(review)
    review_sentences = []
    for sentence in list(review_boundries.sents): # 1, 2
      sentence_aspects = []
      list_dep, list_pos = getDependencies(sentence) # 3
      aspect_opinion_pairs = aspectOpinionPairExtractor(list_dep=list_dep, list_pos=list_pos) # 4
      for as_op_pair in aspect_opinion_pairs:
        sentence_aspects.append(as_op_pair)
      review_sentences.append(sentence_aspects)
    reviews_sentences_aspects.append(review_sentences)
  return reviews_sentences_aspects

In [ ]:
def getDependencies(review_sentence):
  document = client.annotate(str(review_sentence))
  sentence = document.sentence[0]
  dependency_parse = sentence.basicDependencies
  token_dict = {}
  pos_dict = {}
  for i in range(0, len(sentence.token)) :
      token_dict[sentence.token[i].tokenEndIndex] = sentence.token[i].word
      pos_dict[sentence.token[i].word] = sentence.token[i].pos

  # get a list of the dependencies with the words they connect
  list_dep=[]
  for i in range(0, len(dependency_parse.edge)):

      source_node = dependency_parse.edge[i].source
      source_name = token_dict[source_node]

      target_node = dependency_parse.edge[i].target
      target_name = token_dict[target_node]

      dep = dependency_parse.edge[i].dep
      list_dep.append((dep, source_name, target_name))
  # print(list_dep)
  return list_dep, pos_dict

In [ ]:
def aspectOpinionPairExtractor(list_dep, list_pos):
  aspect_opinion_pairs = set()
  visited_dep = [False]*len(list_dep)
  for dep in list_dep:
    if dep[0] == 'nsubj':
      # print("nsubj")
      if "JJ" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #1
        aspect, opinion = nsubjAdj(list_dep, list_pos, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "JJ" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #2
        aspect, opinion = apply_nsubj_second(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #3
        aspect, opinion = opinionVerb(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #4
        aspect, opinion = apply_nsubj_forth(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "NN" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #10
        aspect, opinion = apply_nsubj_ten(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'amod' and visited_dep[list_dep.index(dep)] == False:
      # print("amod")
      if "JJ" in list_pos[dep[2]]:
        aspect, opinion = apply_amod_sixth(list_dep, dep) #6
        # Get multi aspects for same opinion
        aspects = apply_amod_eight(list_dep, aspect) #8
        aspects.append(aspect)
        for a in aspects:
          addPair(a, opinion, list_dep, aspect_opinion_pairs)

        # Get multi opinions for same aspect
        aspect, opinions = multiOpinion(list_dep, dep, visited_dep) #7
        for o in opinions:
          addPair(aspect, o, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'obl' and "JJ" in list_pos[dep[1]]: #9
      # print("obl")
      aspect = dep[2]
      opinion = dep[1]
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'acl:relcl': #5
      # print("acl:relcl")
      aspect, opinion = relativeClause(list_dep, dep)
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
  return aspect_opinion_pairs

In [ ]:
def addPair(aspect, opinion, list_dep, aspect_opinion_pairs):
  if aspect == "" or opinion == "":
    return
  aspect = getCompund(list_dep, aspect)
  opinion = check_neg(list_dep, opinion)
  p = (aspect, opinion)
  # print(p)
  aspect_opinion_pairs.add(p)

## Dependencies Rules

In [ ]:
# Call after any function to return compund aspects or same aspect => battery life
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getCompund(list_dep, aspect):
  for dep in list_dep:
    if dep[0] == "compound" and dep[1] == aspect :
      aspect = dep[2] + " " + aspect
      break;
  return aspect

In [ ]:
# 1
# Extract pair when it's after capular verb => the battery is good 
# and if having comparitave adj then get the advcl => the battery is more satisfying
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# dependency => the nsubj dep that leaded to this case
def nsubjAdj(list_dep, list_pos, dependency):
  aspect = dependency[2]
  opinion = dependency[1]
  if list_pos[dependency[1]] == "JJR":
    for dep in list_dep:
      if dep[0] == "advcl" and dep[1] == dependency[1]:
        opinion = dep[2]
  return aspect, opinion

In [ ]:
# 2
# when the second argument in nsubj dependency is not noun and the first is adj.
# the feature of the sentence will be the object of it.
# use dependencies (nsubj , xcomp , obj) to get the feature.
# Ex : it is great having the LCD Display.
def apply_nsubj_second(list_dep, dependancy):
  f_arg = ''
  s_arg = dependancy[1]
  verb = ''
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == s_arg :
      verb = dep[2]
      break
    if dep[0] == 'dep' and dep[1] == s_arg :
      verb = dep[2]
      break
  for dep in list_dep:
    if dep[0] == 'obj' and dep[1] == verb :
      f_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg

In [ ]:
# 3
# Extract pair when the opinions is the verb like love and like
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# dependency => the current dep that leaded to this case
# sentiment_verbs => list of verbs that may be opinion
sentiment_verbs = ["like", "love", "adore", "enjoyed", "liked", "loved", "enjoy"]
def opinionVerb(list_dep, dep):
  aspect = ""
  opinion = ""
  if(sentiment_verbs.count(dep[1])):
    opinion = dep[1]
    for dep in list_dep:
      if (dep[0] == "obj" and dep[1] == opinion):
        aspect = dep[2]
  return aspect, opinion

In [ ]:
# 4
# when the second argument is noun and the opnion part is verb.
# the opinion will be the complement of the verb.
# Ex : the flash works great.
def apply_nsubj_forth (list_dep, dependency) :
  f_arg = dependency[2]
  s_arg = ''
  verb = dependency[1]
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == verb :
      s_arg = dep[2]
      break
    if dep[0] == 'advmod' and dep[1] == verb :
      s_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [ ]:
# 5
# Extract pair when there is a relative clause relation between the aspect and the opinion => movie mode that works good
# list_dep => dependency_parsed
# dependency => the current dep that leaded to this case
def relativeClause(list_dep, dependency):
  aspect = dependency[1]
  relcl_opinion = dependency[2]
  opinion = ""
  for dep in list_dep:
    if dep[0] == "advmod" and relcl_opinion == dep[1]:
      opinion = dep[2]
      return aspect, opinion
  return aspect, opinion

In [ ]:
# 6
# when the feature and opinion pair candidate could in same dependency.
# amod dependency the first arg is considered to be the feature.
# the second arg is considered to be the opinion words.
# Ex : this is a great screen.
def apply_amod_sixth (list_dep, dependency) :
  f_arg = dependency[1]
  s_arg = dependency[2]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [ ]:
# 7-1
# Extract multi opinions if without "and" word. only with ","
# list_dep => dependency_parsed
# dependency => the current dep that leaded to this case
# visited_dep => to make sure if the amod dependency was considered or not in the main loop
def multiOpinion(list_dep, dependency, visited_dep):
  aspect = dependency[1]
  opinions = []
  for dep in list_dep:
    if(dep[0] == "amod" and dependency[1] == dep[1]):
      opinions.append(dep[2])
      visited_dep[list_dep.index(dep)] = True
  opinions = getAndOpinions(list_dep, opinions)
  return aspect, opinions

# 7-2
# Call after any function to get list of opinions if multi exist with "and" word
# list_dep => dependency_parsed
# opnions => list of opinions "List"
def getAndOpinions(list_dep, opinions):
  for dep in list_dep:
    if dep[0] == "conj" and (opinions.count(dep[1]) != 0):
      opinions.append(dep[2])
    elif dep[0] == "conj" and (opinions.count(dep[2]) != 0):
      opinions.append(dep[1])
  return opinions

In [ ]:
# 8
# when the same opinion word is used to describe more than one feature.
# apply conj dependency on the related features. 
# Ex : Nokia has a good screen and battery.
def apply_amod_eight (list_dep, feature) :
  feats = []
  for dep in list_dep:
    if dep[0] == 'conj' and dep[1] == feature:
      feats.append(dep[2])
  #call the function check compound of each feature
  return feats

In [ ]:
# 9
# Extract pair when theres a preposition => i am happy "with" my phone
# dependency => the current dep that leaded to this case
def prepositions(dependency):
  aspect = dependency[2]
  opinion = dependency[1]
  return aspect, opinion

In [ ]:
# 10
# when nsubj dependency is between two nouns.
# the first argument can be used as a opinion word.
# Ex : the battery was never a problem.
def apply_nsubj_ten (list_dep, dependency):
  f_arg = dependency[2]
  s_arg = dependency[1]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg
  

In [ ]:
# 11
# Call after any function to get list of aspects if multi exist
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getAndAspects(list_dep, aspect):
  aspects = [aspect]
  for dep in list_dep:
    if dep[0] == "conj" and (dep[1] == aspect):
      aspects.append(dep[2])
    elif dep[0] == "conj" and (dep[2] == aspect):
      aspects.append(dep[1])
  for i in range(len(aspects)):
    aspects[i] = getCompund(list_dep, aspects[i])
  return aspects

In [ ]:
# neg
# check negation of the opinion words as it can reverse the sentiment.
def check_neg(list_dep, adj):
    list_neg = ['no' , 'never' , 'not' , 'n\'t' , 'none' , 'neither'] 
    #can also check the negative seed list for that.
    for dep in list_dep:
      if dep[0] == 'advmod' and dep[1] == adj and ( dep[2] in list_neg) :
        adj = dep[2] + ' ' + adj
        return adj
    return adj

## Test

In [ ]:
examples = {1: "The battery life is good",
            2: "It is great having the LCD display",
            3: "I like this camera",
            4: "The optical zoom works great",
            5: "It has movie mode that works good for a digital camera",
            6: "There is a great camera",
            71: "Nokia has fine, excellent, cheapest battery",
            72: "Nokia has fine, excellent and cheapest battery",
            8: "Nokia has good screen and battery",
            9: "I am happy with my Nokia phone",
            10: "Battery life was never a problem for me",
            11: "This camera will give you a great picture quality, LCD screen, and price"}
                        
reviews_list = ["The battery life is good. The screen is bad but the brightness is great. i loved the color",
                "It is great having the LCD display",
                "I like this camera",
                "The optical zoom works great",
                "It has movie mode that works good for a digital camera",
                "There is a great camera",
                "Nokia has fine, excellent, cheapest battery",
                "Nokia has fine, excellent and cheapest battery",
                "Nokia has good screen and battery",
                "I am happy with my Nokia phone",
                "Battery life was never a problem for me",
                "This camera will give you a great picture quality, LCD screen, and price"]
reviews_aspects = reviewsAnalyzer(reviews_list)

In [ ]:
reviews_aspects

[[[('battery life', 'good')],
  [('brightness', 'great'), ('screen', 'bad')],
  [('color', 'loved')]],
 [[('LCD display', 'great')]],
 [[('camera', 'like')]],
 [[('zoom', 'optical'), ('zoom', 'great')]],
 [[('movie mode', 'good'), ('camera', 'digital')]],
 [[('camera', 'great')]],
 [[('Nokia', 'fine'), ('battery', 'excellent'), ('battery', 'cheapest')]],
 [[('Nokia', 'fine'), ('battery', 'excellent'), ('battery', 'cheapest')]],
 [[('battery', 'good'), ('screen', 'good')]],
 [[('Nokia phone', 'happy')]],
 [[('Battery life', 'never problem')]],
 [[('picture quality', 'great'), ('LCD screen', 'great'), ('price', 'great')]]]